In [ ]:
import random
import numpy as np
from numpy import array as a
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

In [ ]:
# Gerando Dados Sintéticos 

# Quantidade de pontos em cada classe
nA, nB, nC = 50, 60, 55

# Posição dos centróides
centroidA, centroidB, centroidC = list(a([[7, 8], [3, 2], [8, 3.5]]))

# Guardando os centróides originais em um DataFrame para visualizar depois
centsDF = pd.DataFrame({'x': [centroidA[0], centroidB[0], centroidC[0]], 
                        'y': [centroidA[1], centroidB[1], centroidC[1]],
                        'classe':['centroid']*3})

# Variâcia de cada classe
varA, varB, varC = .9, 1.1, 1.

# Gerandos aleatoriamente as coordendas x e y dos pontos para as classes A, B e C
A = np.random.randn(nA, 2) * varA + centroidA
B = np.random.randn(nB, 2) * varB + centroidB
C = np.random.randn(nC, 2) * varC + centroidC

# Concatenando as coordenadas dos pontos de A, B e C em uma única matrix "dados"
dados = np.concatenate([A, B, C])

# Criando o vetor com os rótulos de cada ponto
classes = ['A'] * nA + ['B'] * nB + ['C'] * nC

# Criando DataFrame do Pandas com os dados gerados acima
DF = pd.DataFrame({'x': dados[:, 0], 'y': dados[:, 1], 'classe': classes})

# Paleta de cores para os gráficos -> (Ciano, Azul, Verde, Vermelho)
palette = ['#00ffff', '#0000ff', '#00ff00', '#ff0000']

sns.scatterplot(data=pd.concat([DF, centsDF]),
                x='x', 
                y='y', 
                hue='classe', 
                palette=palette)


In [ ]:
# Utilizado o KMeans do SK Learn
kmeans = KMeans(n_clusters=3) # Intanciando um modelo K-Means com 3 clusters
kmeans.fit(dados) # Fazendo o K-Means se ajustar aos nossos dados e tentar encontrar os 3 clusters originais

# Salvando os rótulos encontrados pelo modelo em uma nova coluna do dataframe
DF['sklearn'] = kmeans.labels_ 

# Guardando as coordenadas dos centroids encontrados pelo algoritmo (foi próximo dos originais?)
sklearnDF = pd.DataFrame({'x': kmeans.cluster_centers_[:,0], 
                        'y': kmeans.cluster_centers_[:,1],
                        'sklearn':['centroid']*3})

# Plotando os resultados
plt.figure(figsize=(18,8))
plt.subplot(1,2,1)
sns.scatterplot(data=pd.concat([DF, centsDF]), x='x', y='y', hue='classe', palette=palette)
plt.subplot(1,2,2)
sns.scatterplot(data=pd.concat([DF, sklearnDF]), x='x', y='y', hue='sklearn', palette=palette)

In [ ]:
# Função externa auxiliar para plotar cada iteração do nosso modelo
  # Ela não é necessária, coloquei apenas para facilitar a visualização
def print_steps(dados, labels, centroids):
  plt.figure()
  sns.scatterplot(x=np.concatenate([dados[:,0], centroids[:,0]]), 
                  y=np.concatenate([dados[:,1], centroids[:,1]]), 
                  hue=np.concatenate([labels, [-1]*len(centroids)]),
                  palette=list(reversed(palette))[:len(set(labels))+1])

In [ ]:
class NossoKMeans:
  def __init__(self, n_clusters):
    self.n = n_clusters

  def fit(self, dados):
    # passo 0 -> Inicialização dos clusters e do modelo
      # Dica: para amostrar aleatoriamente pontos do nosso dataset, podemos usar a função random.sample do python
      # por exemplo: centroids = random.sample(list(dados), n_clusters)
    self.cluster_centers_ = a(random.sample(list(dados), self.n))
    self.labels_ = np.zeros(len(dados)).astype(int)

    # Loop
    while True:
      # Função de debug para plotar cada iteração (apenas para ajudar, não é necessária)
      print_steps(dados, self.labels_, self.cluster_centers_)

      # passo 1 -> Atribuir rótulos aos dados conforme centróide mais próximo
        # Para isso, precisamos percorrer todos os dados,
        # comparar a distância com cada um dos centróides e 
        # escolher o index da menor distância euclidiana
      self.labels_ = np.array([np.argmin([np.linalg.norm(p - c) for c in self.cluster_centers_]) for p in dados])

      # passo 2 -> Recalcular centroides a partir dos novos rótulos gerados no passo 1
        # Para isso, iremos computar a média de todos os pontos (dados)
        # pertencentes a cada um dos grupos (relativos aos centróides)
        
      self.cluster_centers_, lastCs = np.array([dados[self.labels_ == i].mean(axis=0) for i in range(self.n)]), self.cluster_centers_

      # passo X -> Condição de parada é quando não há mais alteração da posição dos centroides
        # Para isso, precisamos ter armazenado os valores antigos dos centróides
        # (antes da última alteração) e comparar com o atual. Caso não haja diferença,
        # então podemos quebrar o loop e encerrar nossa função fit()
      if np.linalg.norm(lastCs - self.cluster_centers_) < 1e-9: # poderia fazer == 0, mas como trabalamos com ponto flutuante, as vezes essa igualdade pode falhar em aproximações
        return self

In [ ]:
# Utilizando o nosso modelo de KMeans para agrupar os mesmos dados de maneira idêntica como fizemos com o modelo do SK Learn
nosso = NossoKMeans(n_clusters=3)
nosso.fit(dados)

DF['nosso'] = nosso.labels_

Ms = nosso.cluster_centers_
nossoDF = pd.DataFrame({'x': Ms[:,0], 
                       'y': Ms[:,1],
                       'nosso':['centroid']*3})

# Acompanhe abaixo os plots de cada iteração.
# Observe como os centroides se moveram
# E como os labels de cada ponto do dataset foram se modificando


In [ ]:
# Plotando todos os resultados finais
plt.figure(figsize=(18,8))
plt.subplot(1,3,1)
sns.scatterplot(data=pd.concat([DF, centsDF]), x='x', y='y', hue='classe', palette=palette)
plt.subplot(1,3,2)
sns.scatterplot(data=pd.concat([DF, sklearnDF]), x='x', y='y', hue='sklearn', palette=palette)
plt.subplot(1,3,3)
sns.scatterplot(data=pd.concat([DF, nossoDF]), x='x', y='y', hue='nosso', palette=palette)